In [ ]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
from pathlib import Path

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt

# Basic RNNs

## Generate the Dataset

In [ ]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0,1,n_steps)
    series = 0.5*np.sin((time-offsets1) * (freq1 * 10 +10))
    series += 0.2 * np.sin((time-offsets2) *(freq2 *20+20))
    series += 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
    return series[..., np.newaxis].astype(np.float32)

In [ ]:
np.random.seed(42)

n_steps=50
series=generate_time_series(10000, n_steps+1)
X_train, y_train = series[:7000,:n_steps], series[:7000,-1]
X_valid, y_valid = series[7000:9000,:n_steps], series[7000:9000,-1]
X_test, y_test = series[9000:,:n_steps], series[9000:,-1]


In [ ]:
X_train.shape, y_train.shape

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label ="$t$", y_label = "$x(t)$", legend = True):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bo", label = "Target")
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "rx", markersize=10, label = "Prediction")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize =16)
    if y_label:
        plt.ylabel(y_label, fontsize = 16, rotation=0)
    plt.hlines(0,0,100, linewidth =1)
    plt.axis([0, n_steps+1, -1,1])
    if legend and (y or y_pred):
        plt.legend(fontsize= 14, loc = "upper left")

fig, axes = plt.subplots(nrows=1, ncols=3, sharey =True, figsize=(12,4))
for col in range(3):
    plt.sca(axes[col])
    plot_series(X_valid[col, :, 0], y_valid[col, 0],
                y_label = ("$x(t)$" if col==0 else None), 
                legend = (col==0))


## Computing Some Baselines

In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

In [ ]:
plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0,0])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[50,1]), 
                                 keras.layers.Dense(1)])
model.compile(loss="mse", optimizer='adam')
history = model.fit(X_train,y_train, epochs=20,
                   validation_data= (X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss))+0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss))+1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1,20, 0,0.05])
    plt.legend(fontsize= 14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)
    
plot_learning_curves(history.history["loss"], history.history["val_loss"])

In [ ]:
y_pred = model.predict(X_valid)
plot_series(X_valid[0,:,0], y_valid[0,0], y_pred[0,0])

## Using a Simple RNN

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape= [None, 1])
])

optimizer= keras.optimizers.Adam(learning_rate= 0.005)
model.compile(loss="mse", optimizer = optimizer)
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])

In [ ]:
y_pred=model.predict(X_valid)
plot_series(X_valid[0,:,0], y_valid[0,0], y_pred[0,0])

## Deep RNNs

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape= [None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer = 'adam')
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])

In [ ]:
y_pred=model.predict(X_valid)
plot_series(X_valid[0,:,0], y_valid[0,0], y_pred[0,0])

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape= [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer = 'adam')
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])

In [ ]:
y_pred=model.predict(X_valid)
plot_series(X_valid[0,:,0], y_valid[0,0], y_pred[0,0])

## forecasting Several Steps Ahead

In [ ]:
np.random.seed(43)

series = generate_time_series(1, n_steps +10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X= X_new
for step_ahead in range(10):
    y_pred_one= model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X= np.concatenate([X, y_pred_one], axis=1)

Y_pred = X[:, n_steps:]

In [ ]:
Y_pred.shape

In [ ]:
def plot_multiple_forecasts(X, Y, Y_pred):
    n_steps =X.shape[1]
    ahead=Y.shape[1]
    plot_series(X[0,:,0])
    plt.plot(np.arange(n_steps, n_steps+ahead), Y[0,:,0], "bo-", label = "Actual")
    plt.plot(np.arange(n_steps,n_steps+ahead), Y_pred[0,:,0], "rx-", markersize=10, label = "Forecast")
    plt.axis([0, n_steps+ahead, -1,1])
    plt.legend(fontsize= 14)

plot_multiple_forecasts(X_new, Y_new, Y_pred)


In [ ]:
np.random.seed(42)

n_steps= 50
series = generate_time_series(10000, n_steps +10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:,0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:,0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:,0]

In [ ]:
X = X_valid
for step_ahead in range(10):
    y_pred_one = model.predict(X)[:, np.newaxis, :]
    X= np.concatenate([X, y_pred_one], axis =1)
    
Y_pred =X[:, n_steps:, 0]

In [ ]:
Y_pred.shape

In [ ]:
np.mean(keras.metrics.mean_squared_error(Y_valid, Y_pred))

In [ ]:
Y_naive_pred= np.tile(X_valid[:, -1],10)
np.mean(keras.metrics.mean_squared_error(Y_valid, Y_naive_pred))

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, input_shape= [50, 1]),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer = 'adam')
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape= [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer = 'adam')
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 +10)
X_new, Y_new = series[:, :50,:], series[:, -10:,:]
Y_pred = model.predict(X_new)[..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)

In [ ]:
np.random.seed(42)

n_steps= 50
series = generate_time_series(10000, n_steps +10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y= np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10+1):
    Y[..., step_ahead -1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]


In [ ]:
X_train.shape, Y_train.shape

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape= [None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.TimeDistributesd(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])
model.compile(loss="mse", optimizer = keras.optimizers.Adam(learning_rate= 0.01), metrics =[last_time_step_mse])
history = model.fit(X_train, y_train, epochs =20,validation_data= (X_valid, y_valid))

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 +10)
X_new, Y_new = series[:, :50,:], series[:, 50:,:]
Y_pred = model.predict(X_new)[:,-1][..., np.newaxis]

In [ ]:
plot_multiple_forecasts(X_new, Y_new, Y_pred)

# Deep RNN with Batch Norm